In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd 'drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [3]:
!pip install import-ipynb

In [4]:
import tensorflow as tf
import PIL
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")

import import_ipynb
import prepare_data as pd

importing Jupyter notebook from prepare_data.ipynb


In [21]:
X_train, X_test, y_train, y_test = pd.get_data()

100
200
300
400
500
600
100
200
300
400
500
600


In [7]:
def normalize(data):
    return (data.astype('float32') - 127.5)/127.5 
    
def denormalize(data):
    data = (data + 1) * 127.5
    return data.astype(np.uint8)

In [36]:
w = 852
h = 480
c = 3
shape = (h,w,c)

In [37]:
def res_block_gen(model, kernal_size, filters, strides):    
    gen = model
    
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)

    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
        
    model = add([gen, model])
    
    return model
    
    
class Generator(object):
    def __init__(self, noise_shape):
        
        self.noise_shape = noise_shape
    def generator(self):
        
        gen_input = Input(shape = self.noise_shape)
     
        model = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(gen_input)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
        
        gen_model = model
        
        for index in range(4):
            model = res_block_gen(model, 3, 64, 1)
     
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(model)
        model = BatchNormalization(momentum = 0.5)(model)
        model = add([gen_model, model])
     
        model = Conv2D(filters = c, kernel_size = 9, strides = 1, padding = "same")(model)
        model = Activation('tanh')(model)
    
        generator_model = Model(inputs = gen_input, outputs = model)
        return generator_model

In [38]:
generator = Generator(shape).generator()

In [26]:
generator.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 480, 852, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 480, 852, 64) 15616       input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 480, 852, 64) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 480, 852, 64) 36928       p_re_lu[0][0]                    
______________________________________________________________________________________________

In [28]:
class Discriminator(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape
    
    def discriminator(self):
        
        dis_input = Input(shape = self.image_shape)
        
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(dis_input)
        model = LeakyReLU(alpha = 0.2)(model)

        model = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = "same")(model)
        model = LeakyReLU(alpha = 0.2)(model)

        model = Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = "same")(model)
        model = LeakyReLU(alpha = 0.2)(model)
        
        model = Flatten()(model)
       
        model = Dense(1)(model)
        model = Activation('sigmoid')(model) 
        
        discriminator_model = Model(inputs = dis_input, outputs = model)
        
        return discriminator_model

In [29]:
discriminator = Discriminator(shape).discriminator()
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 480, 852, 3)]     0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 480, 852, 64)      1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 480, 852, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 480, 852, 32)      18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 480, 852, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 480, 852, 16)      4624      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 480, 852, 16)      0   

In [30]:
def define_gan(discriminator, shape, generator, optimizer):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss="binary_crossentropy", loss_weights=[1., 1e-3], optimizer=optimizer)

    return gan

In [31]:
opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
generator.compile(loss="binary_crossentropy", optimizer=opt)
discriminator.compile(loss="binary_crossentropy", optimizer=opt)

gan = define_gan(discriminator, shape, generator, opt)
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 480, 852, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 480, 852, 3)       366147    
_________________________________________________________________
model_1 (Functional)         (None, 1)                 6568241   
Total params: 6,934,388
Trainable params: 364,995
Non-trainable params: 6,569,393
_________________________________________________________________


In [32]:
def load_real_samples(data):
    trainX = data[0:300]
    X = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2], c))
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    return X

In [35]:
def train(g_model, d_model, gan_model, dataset_x, dataset_y, n_epochs=5):
    batch = 3
    bat_per_epo = int(dataset_x.shape[0] / batch)
    g_loss = d_loss1 = d_loss2 = 0
    for i in range(n_epochs):
        print('Epoch: ', i)
        for j in range(bat_per_epo):
            print('\r', 'Bat: ', j, end='')
            discriminator.trainable = True
            
            rand_nums = np.random.randint(0, dataset_x.shape[0], size=batch)
            
            image_batch_hr = dataset_y[rand_nums]
            image_batch_lr = dataset_x[rand_nums]
            generated_images_sr = generator.predict(image_batch_lr)
            
            real_data_Y = np.ones(batch) - np.random.random_sample(batch)*0.2
            fake_data_Y = np.random.random_sample(batch)*0.2
            
            d_loss1 = d_model.train_on_batch(image_batch_hr, real_data_Y)
            d_loss2 = d_model.train_on_batch(generated_images_sr, fake_data_Y)
            
            rand_nums = np.random.randint(0, dataset_x.shape[0], size=batch)
            image_batch_lr = dataset_x[rand_nums]
            image_batch_hr = dataset_y[rand_nums]

            gan_Y = np.ones(batch) - np.random.random_sample(batch)*0.2
        
            discriminator.trainable = False
            g_loss = gan_model.train_on_batch(image_batch_lr, [image_batch_hr, gan_Y])
        print('\n')
        print("d_loss1: ", d_loss1)
        print("d_loss2: ", d_loss2)
        print( "g_loss: ", g_loss)
        
    g_model.save('generator.h5')
    g_model.save_weights('generator_weights.h5')

In [36]:
dataset_x = load_real_samples(X_train) 
dataset_y = load_real_samples(y_train)
train(generator, discriminator, gan, dataset_x, dataset_y, 5)

Epoch:  0
 Bat:  99

d_loss1:  0.2602364718914032
d_loss2:  0.46932268142700195
g_loss:  [-1.2489731311798096, -1.2502827644348145, 1.309652328491211]
Epoch:  1
 Bat:  99

d_loss1:  0.4245476722717285
d_loss2:  0.2942250669002533
g_loss:  [-3.3624870777130127, -3.364544630050659, 2.0571515560150146]
Epoch:  2
 Bat:  99

d_loss1:  0.4328671395778656
d_loss2:  0.39649930596351624
g_loss:  [-2.2867579460144043, -2.28863263130188, 1.874775767326355]
Epoch:  3
 Bat:  99

d_loss1:  0.35819002985954285
d_loss2:  0.35172829031944275
g_loss:  [-1.747890830039978, -1.7493454217910767, 1.454755425453186]
Epoch:  4
 Bat:  99

d_loss1:  0.3511655628681183
d_loss2:  0.5178008675575256
g_loss:  [-2.2547719478607178, -2.2566328048706055, 1.860749363899231]


In [5]:
from tensorflow.keras.models import load_model
model = load_model('generator.h5')
model.load_weights('generator_weights.h5')

In [34]:
x = pd.read_images('in/frames240')
img = denormalize(model.predict(normalize(x)))
save_images(img, 'gan', 0, False)

In [35]:
pd.frames_to_video('out/gan', 'gan')

100
200
300
400
500
600
